In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# Minimizing a <span style="font-variant:small-caps;">Fsm</span>

In [ ]:
%run FixedPoint.ipynb

In [ ]:
def arb(M):
    for x in M:
        return x

The function `cart_prod(A, B)` computes the <em style="color:blue">Cartesian product</em> $A \times B$ of the sets $A$ and $B$ where
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
def cart_prod(A, B):
    return { (x, y) for x in A for y in B }

Given a set `Pairs` of pairs of state that are known to be separable, the function
`separate(Pairs)` computes the set of pairs of states $(q_1, q_2)$ that can be seen to be separable because there is some $c \in \Sigma$ such that 
$$\delta(q_1,c) = p_1, \quad \delta(q_2,c) = p_2 $$
and $p_1$ and $p_2$ are known to be separable.

In [ ]:
def separate(Pairs):
    return { (q1, q2) for q1 in States
                      for q2 in States
                      for c in Sigma 
                      if (delta[q1, c], delta[q2, c]) in Pairs
           }

The function `minimize(A)` takes a deterministic 
<span style="font-variant:small-caps;">Fsm</span> `A` as its input.
Here `A` is a 5-tuple of the form
$$ A = (Q, \Sigma, \delta, q_0, F) $$
The algorithm performs the folllowing steps:
1. All unreachable states are eliminated.
2. All accepting states are separated form all non-accepting states.
3. States are separated as long as possible.
   Two states $p_1$ and $p_2$ are separable if there is a character 
   $c \in \Sigma$ such that 
   $$\delta(p_1,c) = q_1, \quad \delta(p_2,c) = q_2, \quad \textrm{and} \quad
     \mbox{$q_1$ and $q_2$ are separable.}
   $$
4. States that are equivalent are identified and grouped
   in equivalence classes. 

In [ ]:
def minimize(A):
    States, Sigma, delta, q0, Accepting = A
    States    = fixpoint({q0}, lambda q: { delta[q, c] for c in Sigma })
    Separable = cart_prod(States - Accepting, Accepting) | \
                cart_prod(Accepting, States - Accepting)
    
    def separate(Pairs):
        return { (q1, q2) for q1 in States
                          for q2 in States
                          for c in Sigma 
                          if (delta[q1, c], delta[q2, c]) in Pairs
               }
    
    AllSeparable = fixpoint(Separable, separate)
    Equivalent   = cart_prod(States, States) - AllSeparable
    EquivClasses = { frozenset({ p for p in States if (p, q) in Equivalent })
                     for q in States 
                   }
    newQ0        = arb({ M for M in EquivClasses if q0 in M })
    newAccept    = { M for M in EquivClasses if arb(M) in Accepting }   
    newDelta     = {}
    for q in States:
        for c in Sigma:
            p = delta.get((q, c))
            if p != None:
                classOfP = find_equiv(p, EquivClasses)
                classOfQ = find_equiv(q, EquivClasses)
                newDelta[(classOfQ, c)] = classOfP
    return EquivClasses, Sigma, newDelta, newQ0, newAccept

Given a state $p$, find the equivalence class of $p$.

In [ ]:
def find_equiv(p, eqClasses):
    return arb({ C for C in eqClasses if p in C })